**热门超跌上涨**

### 选股
交易日前一天进行选股。

特征为：当日下跌（或者涨幅小于3%），高点应该**5日之前**出现，高点到当日，回撤**2%~10%**。最高点前连续上涨（红盘或大于-0.5%），**累计上涨应大于10%**，注意一定**要有一定的涨幅**。

如2023/03/29 的688313。

选择出来的股票隔日不一定上涨，但一定有一定数量是上涨股票，**实战时应看次日的表现**。

### 实战
策略应该在**10:20**左右开始运行，交易行为发生约在：10:30。

- 20cm票的涨幅不应高于5%，回撤幅度应不大于2%，10cm票涨幅不超过3%，回撤不大于1%，且当前不是最低。最好不要出现绿盘。
- **缓慢上涨**的票大涨概率大，**最高点2%的**，也有一定作为的概率，最高点少于1.5%的，有作为的概率可能低。
- **k线上最近有大涨的**，有作为的概率大。
- 10:20之前**强拉升的**，高度注意回落，缓慢拉升的不在此列。
- 20cm股票，张7到5个点，容易20cm

In [1]:
from datetime import datetime
from winq import *
import hiq_pyfetch as fetch
from winq.selector import *
db = default(log_level='info')

run_task_count = 50

base_path = '/Users/luoguochun/Downloads/stock'

In [15]:
# 清空结果集
import os
import shutil
for p in ['analyze', 'trade']:
    path = os.sep.join([base_path, p])
    files = os.listdir(path)
    for file in files:
        if file.startswith('.'):
            continue
        p = os.sep.join([path, file])
        if os.path.isdir(file):
            print('rm dir: {}'.format(p))
            shutil.rmtree(p)
        else:
            print('rm file: {}'.format(p))
            os.remove(p)
            

In [16]:
# 记得改时间
test_end_date = '20230417'
with_rt_kdata = False

strategy = strategies['test.SuperFalling'](db=db,
                                           min_trade_days=30,
                                        test_end_date=test_end_date, with_rt_kdata = with_rt_kdata,
                                        run_task_count=run_task_count,
                                        load_daily=db.load_stock_daily, 
                                        load_info=db.load_stock_info,
                                        fetch_daily=fetch.fetch_stock_bar
                                       )
await strategy.prepare(max_first_day_rise=3.0,
                       max_first_day_down=-6.0,
                       max_down_as_rise=-1.5,
                       max_meet_high_days=8,
#                        max_meet_high_days=5,
                       min_high_falling=2.0,
                       max_high_falling=10.0,
                       max_turnover=18.0,
                       min_rise_to_high_ndays=10.0,
                       max_rise_unfill_gaps=5.0,
                       sort_by='chg_pct')

data = await strategy.run()

if data is not None and  len(data) > 0:
    test_end_date = strategy.test_end_date.strftime('%Y%m%d')
    
#     path='{}/analyze/supper_falling_{}.txt'.format(base_path, test_end_date)
#     export_codes_to_path(path=path, data=data)
#     print('saved file: {}\nhit data size: {}\n'.format(path, len(data)))
    
    data20cm = data[(data['code'].str.startswith('sh68') | data['code'].str.startswith('sz30')) & ~data['code'].str.contains('ST')]
    path='{}/analyze/supper_falling_{}_20cm.txt'.format(base_path, test_end_date)
    export_codes_to_path(path=path, data=data20cm)
    print('saved file: {}\nhit data size: {}\n'.format(path, len(data20cm)))
    
#     hot_rank = await fetch.fetch_stock_hot_rank(code=list(data['code'].values))
    
#     data_rank500 = hot_rank[hot_rank['rank'] <= 500]
#     path='{}/analyze/supper_falling_{}_rank500.txt'.format(base_path, test_end_date)
#     export_codes_to_path(path=path, data=data_rank500)
#     print('saved file: {}\nhit data size: {}\n'.format(path, len(data_rank500)))
    
#     data_rank1000 = hot_rank[hot_rank['rank'] <= 1000]
#     path='{}/analyze/supper_falling_{}_rank1000.txt'.format(base_path, test_end_date)
#     export_codes_to_path(path=path, data=data_rank1000)
#     print('saved file: {}\nhit data size: {}\n'.format(path, len(data_rank1000)))
    
#     data_rank1500 = hot_rank[hot_rank['rank'] <= 1500]
#     path='{}/analyze/supper_falling_{}_rank1500.txt'.format(base_path, test_end_date)
#     export_codes_to_path(path=path, data=data_rank1500)
#     print('saved file: {}\nhit data size: {}\n'.format(path, len(data_rank1500)))

# data

处理 sh603678(火炬电子): 100%|██████████| 4962/4962 [02:29<00:00, 33.10it/s]] 


saved file: /Users/luoguochun/Downloads/stock/analyze/supper_falling_20230417_20cm.txt
hit data size: 80



# 实战之前应该仔细看，切记仔细看
策略应该在**10:20**左右开始运行，交易行为发生约在：10:30。

上涨时要参考所在的**相关概念**是否有大涨的票~

- 20cm票的涨幅不应高于10%，**5%~7%最佳**，回撤幅度应不大于2%，10cm票涨幅不超过3%，回撤不大于1%，且当前不是最低。最好不要出现绿盘。
- **缓慢上涨**的票大涨概率大，**最高点2%的**，也有一定作为的概率，最高点少于1.5%的，有作为的概率可能低。
- **k线上最近有大涨的**，有作为的概率大。
- 10:20之前**强拉升的**，高度注意回落，缓慢拉升的不在此列。
- **拉高回落**的票，大概率是出货，**不要买不要买不要买**
- 开盘前5分钟，**波动比较大，2%之内都正常**，**不要太着急卖**，太大波动才考虑卖，不要犹豫，狠心卖
- 要参考股吧的人气，**500名以内的大胆上！** 1000名以外的要考虑考虑！
- 考虑所在的行业，**行业有涨幅，股票没涨的**，要大方考虑！
- 自己没勇气上的，很可能是大涨的，一定要认真看行业！
- 不要怕百元股

当和自己预期不一致时，一定要**狠心卖**！


In [14]:
test_end_date = '20230417'
with_rt_kdata = True

pool_date = '20230417'
is_20cm = False

# 记得修改时间
path='{}/analyze/supper_falling_{}.txt'.format(base_path, pool_date)
if is_20cm:
    path='{}/analyze/supper_falling_{}_20cm.txt'.format(base_path, pool_date)


codes=None
with open(path) as f:
    codes = f.read()
    codes = codes.split(',')
    codes = ['sh'+code if code.startswith('6') else 'sz'+code for code in codes]
    
print('loaded file: {}\ndata size: {}'.format(path, len(codes)))


strategy = strategies['test.SimpleRise'](db=db, 
                                        test_end_date=test_end_date, with_rt_kdata = with_rt_kdata,
                                        run_task_count=run_task_count,
                                        load_daily=db.load_stock_daily, 
                                        load_info=db.load_stock_info,
                                        fetch_daily=fetch.fetch_stock_bar
                                       )
await strategy.prepare(cal_time='10:30:00',
                       max_turnover=7.0,
                       max_down_as_rise=-1.0,
#                        max_20cm_rise=5.0,
                       max_20cm_rise=7.0,
                       max_20cm_falling=2.0,
                       max_10cm_rise=3.0,
                       max_10cm_falling=1.50,
                       sort_by='chg_pct')

data = await strategy.run(codes=None)

if data is not None and  len(data) > 0:
    test_end_date = strategy.test_end_date.strftime('%Y%m%d')
    path='{}/trade/simple_rise_{}.txt'.format(base_path, test_end_date)
    if is_20cm:
        path='{}/trade/simple_rise_20cm_{}.txt'.format(base_path, test_end_date)
    export_codes_to_path(path=path, data=data)
data

处理 sz300753(爱朋医疗):   0%|          | 4/4962 [00:00<02:23, 34.52it/s]

loaded file: /Users/luoguochun/Downloads/stock/analyze/supper_falling_20230406.txt
data size: 153


处理 sz002325(洪涛股份): 100%|██████████| 4962/4962 [00:55<00:00, 89.04it/s]   


,code,name
0,sh600550,保变电气
1,sh688630,芯碁微装
2,sh688078,龙软科技
3,sz300900,广联航空
4,sz300790,宇瞳光学
5,sz300853,申昊科技
6,sz002880,卫光生物
7,sz300708,聚灿光电
8,sz001367,C海森药业
9,sh688221,前沿生物


### 以下部分是测试

In [2]:
await fetch.fetch_stock_hot_rank(code=['sz300468', 'sh688387', 'sh688192'])

,code,market_all_count,rank,rank_chang,calc_time
0,sz300468,5155,402,-1,2023-04-13 08:40:00
1,sh688387,5155,1418,-4,2023-04-13 08:40:00
2,sh688192,5155,4599,21,2023-04-13 08:40:00


In [3]:
pool_date = '20230413'
is_20cm = False

# 记得修改时间
path='{}/analyze/supper_falling_{}.txt'.format(base_path, pool_date)
if is_20cm:
    path='{}/analyze/supper_falling_{}_20cm.txt'.format(base_path, pool_date)


codes=None
with open(path) as f:
    codes = f.read()
    codes = codes.split(',')
    codes = ['sh'+code if code.startswith('6') else 'sz'+code for code in codes]
    
print('loaded file: {}\ndata size: {}'.format(path, len(codes)))

loaded file: /Users/luoguochun/Downloads/stock/analyze/supper_falling_20230413.txt
data size: 215


In [4]:
hot_rank = await fetch.fetch_stock_hot_rank(code=codes)
hot_rank

,code,market_all_count,rank,rank_chang,calc_time
0,sh603893,5155,301,0,2023-04-13 08:40:00
1,sh688066,5155,1781,28,2023-04-13 08:40:00
2,sh688262,5155,1427,-7,2023-04-13 08:40:00
3,sh688228,5155,1396,5,2023-04-13 08:40:00
4,sh603171,5155,1752,-17,2023-04-13 08:40:00
5,sh603444,5155,513,1,2023-04-13 08:40:00
6,sh688387,5155,1418,-4,2023-04-13 08:40:00
7,sz300476,5155,202,6,2023-04-13 08:40:00
8,sh688308,5155,4121,23,2023-04-13 08:40:00
9,sh688010,5155,3431,-11,2023-04-13 08:40:00


In [7]:
hot_rank[hot_rank['rank'] < 1000]

,code,market_all_count,rank,rank_chang,calc_time
0,sh603893,5155,301,0,2023-04-13 08:40:00
5,sh603444,5155,513,1,2023-04-13 08:40:00
7,sz300476,5155,202,6,2023-04-13 08:40:00
10,sz300353,5155,630,-6,2023-04-13 08:40:00
12,sh603650,5155,448,0,2023-04-13 08:40:00
13,sz300413,5155,418,3,2023-04-13 08:40:00
15,sh600584,5155,126,-1,2023-04-13 08:40:00
17,sz300661,5155,412,12,2023-04-13 08:40:00
18,sh600703,5155,136,0,2023-04-13 08:40:00
19,sh600271,5155,383,0,2023-04-13 08:40:00


In [11]:
#hot_rank = await fetch.fetch_stock_hot_rank(code=list(data20cm['code'].values))
data_rank1500 = hot_rank[hot_rank['rank'] <= 500]
path='{}/analyze/supper_falling_{}_20cm_rank500.txt'.format(base_path, test_end_date)
export_codes_to_path(path=path, data=data_rank1500)
print('saved file: {}\nhit data size: {}\n'.format(path, len(data_rank1500)))

saved file: /Users/luoguochun/Downloads/stock/analyze/supper_falling_20230413_20cm_rank500.txt
hit data size: 13



In [12]:
data_rank1500 = hot_rank[hot_rank['rank'] <= 1000]
len(data_rank1500)

25